In [12]:
#import all necessary packages
import pandas as pd
import numpy as np

In [13]:
#importing data
df_all = pd.read_csv("dataset.csv")

In [ ]:
# rename the 'Price' column to 'Date'
df_all.rename(columns={'Price': 'Date'}, inplace=True)

# Convert 'Date' to datetime
df_all['Date'] = pd.to_datetime(df_all['Date'], errors='coerce')

# Set the 'Date' column as index
df_all.set_index('Date', inplace=True)

# Check the head and index type
print(df_all.head())
print(df_all.index)

                          gold            usd_inr                dxy  \
Date                                                                   
NaT                       GC=F           USDINR=X           DX-Y.NYB   
NaT                        NaN                NaN                NaN   
2015-01-02              1186.0  63.02799987792969  91.08000183105469   
2015-01-05  1203.9000244140625  63.27799987792969  91.37999725341797   
2015-01-06   1219.300048828125   63.3380012512207               91.5   

                treasury_yield               sp500                 vix  \
Date                                                                     
NaT                       ^TNX               ^GSPC                ^VIX   
NaT                        NaN                 NaN                 NaN   
2015-01-02   2.122999906539917   2058.199951171875  17.790000915527344   
2015-01-05  2.0390000343322754  2020.5799560546875  19.920000076293945   
2015-01-06  1.9630000591278076  2002.6099853515625 

/var/folders/vq/x_32pj256zs5w0j98mhvmdyw0000gn/T/ipykernel_21585/3941061031.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_all['Date'] = pd.to_datetime(df_all['Date'], errors='coerce')


In [15]:
# Convert all other columns to numeric
for col in df_all.columns:
    if col != 'Date':
        df_all[col] = pd.to_numeric(df_all[col], errors='coerce')

In [16]:
# Create target variable: next-day gold price
df_all['target'] = df_all['gold'].shift(-1)

# Lag configuration per feature
lags = {
    'gold': [1, 2, 3, 5],
    'usd_inr': [1, 3],
    'dxy': [1, 2],
    'treasury_yield': [1],
    'sp500': [1, 3, 5],
    'vix': [1, 2],
    'crude': [1, 3],
    'silver': [1, 2],
    'btc': [1, 3]
}

# Add lag features
for col, lag_days in lags.items():
    for lag in lag_days:
        df_all[f'{col}_lag{lag}'] = df_all[col].shift(lag)

# Rolling features (applied to gold only, can extend to others if needed)
rolling_windows = [5, 10]
for window in rolling_windows:
    df_all[f'gold_ma{window}'] = df_all['gold'].rolling(window).mean()
    df_all[f'gold_std{window}'] = df_all['gold'].rolling(window).std()

# Ratio features
df_all['gold_silver_ratio'] = df_all['gold'] / df_all['silver']
df_all['gold_crude_ratio'] = df_all['gold'] / df_all['crude']
df_all['gold_dxy_ratio'] = df_all['gold'] / df_all['dxy']
df_all['gold_btc_ratio'] = df_all['gold'] / df_all['btc']

# Final cleanup
df_all.dropna(inplace=True)  # drop rows with NaNs caused by shift/rolling

# View result
print(df_all.head())

                   gold    usd_inr        dxy  treasury_yield        sp500  \
Date                                                                         
2015-01-15  1264.699951  62.088001  92.349998           1.775  1992.670044   
2015-01-16  1276.900024  62.057999  92.519997           1.815  2019.420044   
2015-01-20  1294.199951  61.638000  93.050003           1.807  2022.550049   
2015-01-21  1293.699951  61.647999  92.900002           1.853  2032.119995   
2015-01-22  1300.699951  61.553001  94.239998           1.896  2063.149902   

                  vix      crude     silver         btc       target  ...  \
Date                                                                  ...   
2015-01-15  22.389999  46.250000  17.073000  209.843994  1276.900024  ...   
2015-01-16  20.950001  48.689999  17.739000  208.097000  1294.199951  ...   
2015-01-20  19.889999  46.389999  17.947001  211.315002  1293.699951  ...   
2015-01-21  18.850000  47.779999  18.179001  226.897003  1300.699951

In [18]:
df_all.columns

Index(['gold', 'usd_inr', 'dxy', 'treasury_yield', 'sp500', 'vix', 'crude',
       'silver', 'btc', 'target', 'gold_lag1', 'gold_lag2', 'gold_lag3',
       'gold_lag5', 'usd_inr_lag1', 'usd_inr_lag3', 'dxy_lag1', 'dxy_lag2',
       'treasury_yield_lag1', 'sp500_lag1', 'sp500_lag3', 'sp500_lag5',
       'vix_lag1', 'vix_lag2', 'crude_lag1', 'crude_lag3', 'silver_lag1',
       'silver_lag2', 'btc_lag1', 'btc_lag3', 'gold_ma5', 'gold_std5',
       'gold_ma10', 'gold_std10', 'gold_silver_ratio', 'gold_crude_ratio',
       'gold_dxy_ratio', 'gold_btc_ratio'],
      dtype='object')